In [ ]:
from threading import Thread

# 語法
# new_thread = Thread(target=fn, args=args_tuple, name=None)


## 建立&執行thread

In [1]:
# current_thread：獲取目前執行的執行緒。
# Thread：建立新執行緒的類別。
# sleep：讓程式暫停執行一段時間（單位為秒）。
# perf_counter：精確地計算程式執行的時間，適合測量短時間的事件。
from threading import current_thread, Thread
from time import sleep, perf_counter

def print_hello():
    sleep(2)  # 暫停 2 秒
    print(f"{current_thread().name}: Hello")

def print_message(msg):
    sleep(1)  # 暫停 1 秒
    print(f"{current_thread().name}: {msg}")


In [4]:
# 紀錄程式開始時間
start = perf_counter()

# 建立執行緒
t1 = Thread(target=print_hello, name="Th01")
t2 = Thread(target=print_hello, name="Th02")
t3 = Thread(target=print_message, args=("Good morning",), name="Th03") # args：指定傳給函式的參數（必須用 tuple）。

# 啟動執行緒
t1.start()
t2.start()
t3.start()

# 等待執行緒結束
t1.join()
t2.join()
t3.join()

# 計算總執行時間
elapsed = perf_counter() - start
print(f"elapsed: {elapsed:.2f} sec")


Th03: Good morning
Th02: Hello
Th01: Hello
elapsed: 2.00 sec


## 守護thread(daemon thread)
- 守護執行緒 (Daemon Thread) 是一種在背景中執行的執行緒，它的特點是當主程式結束時，守護執行緒會自動終止而不會等待它執行完畢。
- 如果你希望執行緒執行完畢後主程式再結束，請使用 join()。
- 如果你希望主程式結束時自動終止執行緒，不用等待它執行完畢，可以設置成 守護執行緒 (daemon=True)。

In [ ]:
from threading import current_thread, Thread
from time import sleep, perf_counter

def daemon_func():
    sleep(3)
    print(f"{current_thread().name}: Hello from daemon")

def func():
    sleep(1)
    print(f"{current_thread().name}: Hello from non-daemon")

# 建立守護執行緒
t1=Thread(target=daemon_func, name="Daemon Thread", daemon=True)

#建立一般執行緒
t2=Thread(target=func, name="Non-Daemon Thread")

t1.start()
t2.start()
print("Exiting the main program")



Exiting the main program


Non-Daemon Thread: Hello from non-daemon
Daemon Thread: Hello from daemon


## 建立執行續類別

In [1]:
import requests
from threading import Thread

class HttpRequestThread(Thread): # 繼承threading.Thread
    def __init__(self, url):
        super().__init__()
        self.url = url
    def run(self):
        print(f"確認 {self.url}...")
        try:
            response = requests.get(self.url)
            pokemon = response.json()
            print(pokemon["name"])
        except Exception as e:
            print(e)

def main():
    urls = [
        "https://pokeapi.co/api/v2/pokemon/10", "https://pokeapi.co/api/v2/pokemon/20", "https://pokeapi.co/api/v2/pokemon/30"
    ]

     # 建立執行緒
    threads = [HttpRequestThread(url) for url in urls]

    # 啟動每個執行緒
    for thread in threads:
        thread.start()

    # 等待所有執行緒完成
    for thread in threads:
        thread.join()

main()

確認 https://pokeapi.co/api/v2/pokemon/10...
確認 https://pokeapi.co/api/v2/pokemon/20...
確認 https://pokeapi.co/api/v2/pokemon/30...
caterpie
nidorina
raticate


## 執行續池(threed pool)
- 是一種在程式中實現平行執行的模式。
- 每一個在池中的執行緒稱爲「worker」
- ==執行緒池允許我們配置worker執行緒的數量，並爲每個worker執行緒提供特定的命名約定。==

In [2]:
from time import sleep, perf_counter
from concurrent. futures import ThreadPoolExecutor

def task(id):
    print(f"Starting the task {id} ... ")
    sleep(1)
    return f"Done with task {id}"

with ThreadPoolExecutor() as executor:
    f1 = executor.submit(task, 1)
    f2 = executor.submit(task, 2)
    print(f1.result())
    print(f2.result())

Starting the task 1 ... 
Starting the task 2 ... 
Done with task 1
Done with task 2


## 同步執行緒(lock)
- 同步執行緒是一種機制，可確保二個或多個執行緒不會同時執行程式的共享資料區塊。
```
# 建立lock類別
lock = Lock()

# lock鎖定
lock.acquire()

# lock解放
lock.release()
```

In [8]:
from threading import Lock, Thread as Thread
import requests
from time import sleep

urls = [
        "https://pokeapi.co/api/v2/pokemon/10", "https://pokeapi.co/api/v2/pokemon/20", "https://pokeapi.co/api/v2/pokemon/30"
    ]

x=0
mylock = Lock()

def get_pokemon(lock, url):
    global x
    try:
        # 模擬共用資源
        lock.acquire()
        local_x = x
        local_x+=1  
        sleep(1)
        x = local_x
        lock.release()
        response = requests.get(url)
        pokemon = response.json()
        print(pokemon["name"])
    except Exception as e:
        print(e)

t1=Thread(target=get_pokemon, args=(mylock,urls[0]))
t2=Thread(target=get_pokemon, args=(mylock,urls[1]))
t3=Thread(target=get_pokemon, args=(mylock,urls[2]))
t1.start()
t2.start()
t3.start()
t1.join()
t2.join()
t3.join()
print(f"final value of x: {x}")




caterpie
raticate
nidorina
final value of x: 3


## 使用queue交換資料
python的queue有3種類型：
-  queue.Queue(maxsize=0)：
    - 先進先出(FIFO)隊列
    - maxsize 參數可限制放置項目的數量，若設為 0 表示無上限。
- queue.LifoQueue(maxsize=0)：
    - 後進先出(LIFO)隊列，像堆疊一樣。
    - maxsize 參數可限制放置項目的數量，若設為 0 表示無上限。
- queue.PriorityQueue(maxsize=0)
    - 優先級隊列，數值越小的項目會先被取出。

In [9]:
import queue

# 基本操作
q = queue.Queue()
q.put(1)
q.put(2)
q.put(3)

print(q.empty())
print(q.get())
print(q.get())
print(q.get())
print(q.empty())

False
1
2
3
True


## 在不同thread中交換資料



In [ ]:
import queue
from threading import  Thread as Thread
import threading
from time import sleep

# 使用 Python 的 queue 模組和多執行緒 threading 模組來模擬多個工作者 (threads) 一起處理任務的情境

class Worker(Thread):
    def __init__(self, name, q):
        super().__init__()
        self.name = name
        self.queue = q
    def run(self):
        while True:
            item = self.queue.get()
            sleep(1)
            try:
                print(f"{self.name}: {item}")
            finally:
                self.queue.task_done()# 在完成一項工作後,此函式向工作已完成的queue發送一個訊號。

my_queue=queue.Queue()
for i in range(10):
    my_queue.put(f"task {i+1}")

# 建立5個thread
for i in range(5):
    worker = Worker(f"thread{i+1}", my_queue)
    worker.daemon = True
    worker.start()

my_queue.join()




thread1: task 1
thread4: task 4
thread5: task 5
thread2: task 2
thread3: task 3
thread4: task 7
thread2: task 9
thread5: task 8
thread1: task 6
thread3: task 10


## 生產者-消費者模式

In [14]:
import queue
import time
import threading

# 建立一個隊列，最大容量為5
q = queue.Queue(5)

# 生產者類別，繼承自 threading.Thread
class Producer(threading.Thread):
    def __init__(self, thread_name):
        super().__init__()
        self.name = thread_name
        self.count = 1

    def run(self):
        while True:
            if q.full():
                print('queue is full')
                break
            else:
                msg = str(self.count)
                q.put(msg)
                print(self.name + ' put ' + msg + ', qsize: ' + str(q.qsize()))
                self.count += 1
            time.sleep(1)

# 消費者類別，繼承自 threading.Thread
class Consumer(threading.Thread):
    def __init__(self, thread_name):
        super().__init__()
        self.name = thread_name

    def run(self):
        while True:
            if q.empty():
                print('queue is empty')
                break
            else:
                msg = q.get()
                print(self.name + ' get ' + msg + ', qsize: ' + str(q.qsize()))
                time.sleep(2)

# 創建並啟動生產者與消費者執行緒
p = Producer('producer')
p.start()

c = Consumer('consumer')
c.start()

# 等待生產者和消費者執行緒結束
p.join()
c.join()


producer put 1, qsize: 1
consumer get 1, qsize: 0
producer put 2, qsize: 1
producer put 3, qsize: 2
consumer get 2, qsize: 1
producer put 4, qsize: 2
producer put 5, qsize: 3consumer get 3, qsize: 2

producer put 6, qsize: 3
consumer get 4, qsize: 2producer put 7, qsize: 3

producer put 8, qsize: 4
consumer get 5, qsize: 3
producer put 9, qsize: 4
producer put 10, qsize: 5
consumer get 6, qsize: 4
producer put 11, qsize: 5
queue is full
consumer get 7, qsize: 4
consumer get 8, qsize: 3
consumer get 9, qsize: 2
consumer get 10, qsize: 1
consumer get 11, qsize: 0
queue is empty


## 多程序
- 程序試指電腦中執行的實例。
- 同一個程式備執行多次，就會有多個程序。
- 一個具有四核的CPU，同時可以執行四個程序。

- 針對多程序程式設計,Python提供了multiprocessing套件,此套件包含二種實現多程序的方法。
    - 使用Process物件
    - 使用Pool物件


![upgit_20241102_1730550615.png](https://raw.githubusercontent.com/kcwc1029/obsidian-upgit-image/main/2024/11/upgit_20241102_1730550615.png)


### 以一般Python程式來處理二個計算任務

In [ ]:
# NOTE: 要用py跑
from time import perf_counter

def task1():
    result = 0
    for i in range(10**8):
        result += 1
    return result

def task2():
    result = 0
    for i in range(10**8):
        result += 2
    return result

# 開始計時
start = perf_counter()

# 執行任務
result1 = task1()
result2 = task2()
result3 = result1 + result2

# 顯示結果
print(f"result3 = {result3}")

# 計算並顯示執行時間
elapsed = perf_counter() - start
print(f"花費時間: {elapsed:.2f} sec")


result3 = 300000000
花費時間: 8.10 sec


### 使用Process 物件執行耗時計算
- Process物件提供了與Thread物件相同的API。
- 使用start()方法來啓動程序。
- 若要等待程序完成工作,可以呼叫join()方法。

In [1]:
from time import perf_counter
import multiprocessing

def task1():
    result = 0
    for i in range(10**8):
        result += 1

def task2():
    result = 0
    for i in range(10**8):
        result += 2

if __name__ == '__main__':
    # 計時開始
    start = perf_counter()

    # 建立兩個進程
    p1 = multiprocessing.Process(target=task1)
    p2 = multiprocessing.Process(target=task2)

    # 啟動進程
    p1.start()
    p2.start()

    # 等待進程完成
    p1.join()
    p2.join()

    # 計算並顯示執行時間
    elapsed = perf_counter() - start
    print(f"花費時間: {elapsed:.2f} sec")


花費時間: 0.07 sec


In [ ]:
# 上述程式並無法取得結果，如果要取得結果，就需要依靠multiprocessing模組提供的queue
from time import perf_counter
import multiprocessing

def task1(queue):
    result = 0
    for i in range(10**2):
        result += 1
    queue.put(result)

def task2(queue):
    result = 0
    for i in range(10**2):
        result += 2
    queue.put(result)  # 把這行放到迴圈外，避免重複放入 queue

if __name__ == "__main__":
    queue = multiprocessing.Queue()
    start = perf_counter()

    p1 = multiprocessing.Process(target=task1, args=(queue,))
    p2 = multiprocessing.Process(target=task2, args=(queue,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    result = queue.get() + queue.get()
    print(f"result= {result}")

    elapsed = perf_counter() - start
    print(f"花費時間: {elapsed:.2f} sec")


## 多程序進行批量處理圖像縮圖

In [1]:
!pip install pillow

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# ERROR: 可以跑，但無法吃到圖片
import multiprocessing
import time
import os
from PIL import Image, ImageFilter

# 定義圖片來源和縮圖儲存的資料夾
source_dir = ""
thumb_dir = "./temp image/thumb dir"


# 建立縮圖函數
def create_thumbnail(filename, size=(50, 50), thumb_dir=thumb_dir):
    try:
        img = Image.open(filename)  # 開啟圖片檔案
        img.show()  # 顯示圖片
        img = img.filter(ImageFilter.GaussianBlur)  # 應用高斯模糊濾鏡
        img.thumbnail(size)  # 調整圖片大小
        img.save(f"{thumb_dir}/{os.path.basename(filename)}")  # 儲存縮圖
        print(f"{filename} was processed")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

if __name__ == '__main__':
    # 確認縮圖資料夾是否存在，若不存在則建立
    if not os.path.exists(thumb_dir):
        os.makedirs(thumb_dir)

    # 自動取得圖片來源資料夾中的所有圖片檔案路徑
    filenames = [os.path.join(source_dir, file) for file in os.listdir(source_dir) if file.endswith('.jpg')]

    # 記錄開始時間
    start = time.perf_counter()

    # 建立進程列表
    processes = []
    for f in filenames:
        p = multiprocessing.Process(target=create_thumbnail, args=(f,))
        processes.append(p)

    # 啟動所有進程
    for p in processes:
        p.start()

    # 等待所有進程完成
    for p in processes:
        p.join()

    # 計算並輸出執行時間
    elapsed = time.perf_counter() - start
    print(f"elapsed: {elapsed:.2f} sec")


elapsed: 0.13 sec


### 使用Pool 物件執行耗時計算

In [ ]:
# NOTE: 要用py跑
from multiprocessing import Pool
from time import perf_counter

# 定義計算任務
def task(x):
    result = 0
    # 對於 10**8 的範圍進行累加計算
    for i in range(10**8):
        result += x
    return result

if __name__ == '__main__':
    start = perf_counter()  # 計時開始

    # 使用多進程池 (Pool) 來處理任務
    with Pool(3) as p:  # 建立 3 個進程
        # 使用 map 方法將任務分配給每個進程，並將結果打印出來
        print(p.map(task, [1, 2, 3]))

    # 計算並輸出執行時間
    elapsed = perf_counter() - start
    print(f"elapsed: {elapsed:.2f} sec")

## 程序池(Process pool)
- 也是一種在程式中實現平行運算的模式，允許我們有效率地自動管理池中的程序，並允許我們配置程序的數量。
- 要建立程序池,我們要使用Python的ProcessPoolExecutor類別,此類別繼承自Executor類別,有三個方法可控制程序池：
    - submit()：呼叫一個要執行的函式,並回傳Future物件。此方法接受一個函式並以非同步方式執行。
    - map()：爲迭代中的每一個成員非同步執行函式。
    - shutdown()：關閉executor
- 當我們建立一個ProcessPoolExecutor實例後，,Python即會啓動Executor，並會回傳Future物件，表示非同步操作的最終結果。
- Future類別有二個有用的方法：
    - result()：回傳非同步操作的結果。
    - exception()：若發生例外,則回傳非同步操作的例外。

In [ ]:
import time
import os
from PIL import Image, ImageFilter
from concurrent. futures import ProcessPoolExecutor

# 這邊有餘力再回來補

## 程序間共享記憶體
- process與thread的最大不同是每個process皆擁有獨立的記憶體空間，不像多個thread會共享一個process的記憶體空間。
- 因此當我們在設計多程序程式時，一般會以Queue來交換資料。
- 除了以Queue來交換資料外，若我們真的有需要讓多個程序共享記憶體空間，multiprocessing模組也提供了共享記憶體的方法，讓我們可以使用Value或Array管理共享記憶體：
    - multiprocessing.Value：管理共享值
    - multiprocessing.Array：管理共享陣列值

In [ ]:
# NOTE: 要用py跑
from multiprocessing import Process, Value, Array, Lock
from time import sleep

# 定義第一個任務
def task1(lock, n, a):
    lock.acquire()  # 獲取鎖
    num1 = n.value
    num1 += 100
    sleep(0.2)
    n.value = num1

    for i in range(len(a)):
        a[i] = a[i] * a[i]
    lock.release()  # 釋放鎖

# 定義第二個任務
def task2(lock, n, a):
    lock.acquire()  # 獲取鎖
    num2 = n.value
    num2 += 200
    sleep(0.2)
    n.value = num2

    for i in range(len(a)):
        a[i] = -a[i]
    lock.release()  # 釋放鎖

if __name__ == '__main__':
    lock = Lock()
    num = Value('d', 0.0)  # 定義共享變數，double 類型
    arr = Array('i', range(10))  # 定義共享陣列，int 類型

    # 創建進程
    p1 = Process(target=task1, args=(lock, num, arr))
    p2 = Process(target=task2, args=(lock, num, arr))

    # 啟動進程
    p1.start()
    p2.start()

    # 等待進程結束
    p1.join()
    p2.join()

    # 輸出結果
    print(f"num = {num.value}")
    print(f"arr = {arr[:]}")


num = 0.0
arr = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
